# Practical Exercice: Global bandwidth selection

The aim of this practical exercise is to estimate the optimal global bandwidth  for local linear regression using the Asymptotic Mean Integrated Squared Error (AMISE) criterion. 
The theoretical expression of the AMISE, optimal bandwidth under homoscedasticity and a quartic (biweight) kernel is given by

$$
h_{\mathrm{AMISE}} \;=\; n^{-1/5}\left( \frac{35 \, \sigma^2 \, \lvert \mathrm{supp}(X)\rvert}{\theta_{22}} \right)^{1/5} $$

where $\sigma^2 = \mathrm{Var}(\varepsilon)$ and $\theta_{22} \;=\; \int \big(m''(x)\big)^2 f_X(x)\, dx$.

Since both $\sigma^2$ and $\theta_{22}$ are unknown, they must be estimated from the data. Following the course notes, the strategy is to split the sample into $N$ blocks and, in each block, fit a quartic regression model of the form 
$$y_i = \beta_{0j} + \beta_{1j} x_i + \beta_{2j} x_i^2 + \beta_{3j} x_i^3 + \beta_{4j} x_i^4 + \varepsilon_i
$$ 
where $x_i$ and $y_i$ are the observations belonging to block $j$.

From these blockwise polynomial fits, we obtain estimates of the regression function $\hat m_j(x)$ and  its second derivative $\hat m_j''(x)$. These are then used to form the plug-in estimators

- $\widehat{\theta}_{22}(N) = \frac{1}{n} \sum_{i=1}^n \sum_{j=1}^N 
\Big( \hat m_j''(X_i) \Big)^2 \,\mathbf{1}_{\{X_i \in \mathcal{X}_j\}}$

- $\hat\sigma^2(N) = \frac{1}{n - 5N} \sum_{i=1}^n \sum_{j=1}^N 
\big( Y_i - \hat m_j(X_i) \big)^2 \,\mathbf{1}_{\{X_i \in \mathcal{X}_j\}}.
$


These plug-in estimates are then substituted into the formula of $h_{\mathrm{AMISE}}$  to yield a data-driven bandwidth choice. 

In this simulation study, we generate data with 
- covariates $X \sim \mathrm{Beta}(\alpha,\beta)$ and 
- responses  $Y = m(X) + \varepsilon$, where $m(x) = \sin\!\Big( \big(\tfrac{x}{3} + 0.1\big)^{-1} \Big), \varepsilon \sim \mathcal{N}(0,\sigma^2)$.

We fix $\sigma^2 = 1$ and explore how the estimated bandwidth $\hat h_{\mathrm{AMISE}}$ behaves 
as a function of the sample size $n$, the block size $N$, and the Beta distribution parameters $(\alpha,\beta)$. 
The results are summarised in the figure **global\_bandwidth\_selection.png**, 
which provides a visual assessment of the influence of these parameters on the global bandwidth choice.

## Importing librairies

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Model & simulation

In [2]:
def m_true(
        x: np.ndarray
    ) -> np.ndarray:
    """
    Regression function m(x) = sin( (x/3 + 0.1)^(-1) ).

    Parameters
        - x: Points in [0, 1]

    Returns
        - m(x) at each entry of x (np.ndarray)
        
    """
    return np.sin(1.0 / (x / 3.0 + 0.1))

In [3]:
def simulate(
        n: int, 
        alpha: float, 
        beta: float, 
        sigma: float, 
        rng: np.random.Generator
    ) -> tuple[np.ndarray, np.ndarray]:
    """
    Simulate (X, Y) with X ~ Beta(alpha, beta), Y = m(X) + eps, eps ~ N(0, sigma^2)
    """
    X = rng.beta(alpha, beta, size=n)
    eps = rng.normal(0.0, sigma, size=n)
    Y = m_true(X) + eps
    return X, Y

##  Blocking & OLS

In [4]:
def make_blocks(
        X: np.ndarray, 
        N: int
    ) -> list[np.ndarray]:
    """
    Partition indices {0,...,n-1} into N contiguous blocks in ascending order of X

    Parameters
        - X: Covariate values
        - N: Number of blocks 

    Returns
        - Each array contains the indices belonging to block j (list of np.ndarray)        
    """
    n = X.size
    order = np.argsort(X)
    splits = np.array_split(order, N)
    return [np.sort(b) for b in splits]




In [5]:
def design_matrix_quartic(
        x: np.ndarray
    ) -> np.ndarray:
    """
    Build [1, x, x^2, x^3, x^4]

    Returns
        - np.ndarray of shape (len(x), 5)
    """
    return np.column_stack([np.ones_like(x), x, x**2, x**3, x**4])

In [6]:
def poly_fit_and_eval(
        block_x: np.ndarray, 
        block_y: np.ndarray, 
        eval_x: np.ndarray
    ):
    """
    Fit quartic OLS y ~ [1, x, x^2, x^3, x^4] on (block_x, block_y),
    evaluate m_hat and m_hat'' at eval_x

    Returns
        - m_hat_eval: Fitted function (np.ndarray)
        - m2_hat_eval: Second derivative at eval_x (np.ndarray)
    """
    Xd = design_matrix_quartic(block_x)
    
    # OLS via least squares
    beta, *_ = np.linalg.lstsq(Xd, block_y, rcond=None)
    
    # Evaluate fitted polynomial and its second derivative
    e = design_matrix_quartic(eval_x) @ beta
    
    # For p(x) = b0 + b1 x + b2 x^2 + b3 x^3 + b4 x^4 :
    # p''(x) = 2 b2 + 6 b3 x + 12 b4 x^2
    m2 = 2.0 * beta[2] + 6.0 * beta[3] * eval_x + 12.0 * beta[4] * (eval_x**2)
    return e, m2


## Plug-in estimators

In [7]:
def theta22_hat(
        X: np.ndarray, 
        Y: np.ndarray, 
        N: int
    ) -> float:
    """
    Estimate theta22_hat

    Parameters
        - X, Y: Data 
        - N : Number of blockes

    Returns
        - Estimate of theta_22 (float)
    """
    blocks = make_blocks(X, N)
    n = X.size
    acc = 0.0
    for idx in blocks:
        xb, yb = X[idx], Y[idx]
        m_hat, m2_hat = poly_fit_and_eval(xb, yb, xb)
        acc += np.sum(m2_hat * m2_hat)
    return acc / n


In [8]:
def sigma2_hat(
        X: np.ndarray, 
        Y: np.ndarray, 
        N: int
    ) -> float:
    """
    Estimate of sigma^2
    The denominator uses (n - number_of_parameters_per_block * N) = (n - 5N), since each quartic regression fits 5 coefficients

    Returns
        - Estimate of sigma^2 (float)
    """
    blocks = make_blocks(X, N)
    n = X.size
    rss = 0.0

    for idx in blocks:
        xb, yb = X[idx], Y[idx]
        m_hat, _ = poly_fit_and_eval(xb, yb, xb)
        rss += np.sum((yb - m_hat)**2)
    denom = n - 5 * N

    if denom <= 0: # fall back to simple (n) to avoid division by zero
        denom = n

    return rss / denom

In [9]:
def h_AMISE_hat(
        n: int, 
        sigma2: float, 
        theta22: float, 
        support_len: float = 1.0
    ) -> float:
    """
    Plug into h_AMISE
    For Beta(a,b), |supp(X)| = 1.
    """
    val = (35.0 * sigma2 * support_len) / max(theta22, 1e-12)
    return (n ** (-1.0/5.0)) * (val ** (1.0/5.0))

## Experiments & plots

In [ ]:
def run_experiment(
    alpha_list=(2.0, 2.0),
    beta_list=(2.0, 5.0),
    n_list=(200, 400, 800, 1600),
    N_list=(1, 2, 3, 4, 5),
    sigma=1.0,
    seed=517,
    ):
    """
    Sweep over n, N, and (alpha,beta) and plot h_AMISE_hat.
    """
    rng = np.random.default_rng(seed)

    # h_AMISE vs n (fix alpha,beta,N)
    a_fix, b_fix, N_fix = 2.0, 2.0, 3
    h_vs_n = []
    for n in n_list:
        X, Y = simulate(n, a_fix, b_fix, sigma, rng)
        th22 = theta22_hat(X, Y, N_fix)
        s2 = sigma2_hat(X, Y, N_fix)
        h_vs_n.append(h_AMISE_hat(n, s2, th22, 1.0))
    h_vs_n = np.array(h_vs_n)

    # h_AMISE vs N (fix alpha,beta,n)
    n_fix = 800
    Xb, Yb = simulate(n_fix, a_fix, b_fix, sigma, rng)
    h_vs_N = []
    valid_N = [N for N in N_list if 5 * N < n_fix]
    for N in valid_N:
        th22 = theta22_hat(Xb, Yb, N)
        s2 = sigma2_hat(Xb, Yb, N)
        h_vs_N.append(h_AMISE_hat(n_fix, s2, th22, 1.0))
    h_vs_N = np.array(h_vs_N)

    # h_AMISE vs Beta shape (vary alpha,beta), fix n, N
    n_fix2, N_fix2 = 800, 3
    pairs = list(zip(alpha_list, beta_list))
    h_vs_shape = []
    labels = []
    for (a, b) in pairs:
        Xs, Ys = simulate(n_fix2, a, b, sigma, rng)
        th22 = theta22_hat(Xs, Ys, N_fix2)
        s2 = sigma2_hat(Xs, Ys, N_fix2)
        h_vs_shape.append(h_AMISE_hat(n_fix2, s2, th22, 1.0))
        labels.append(f"α={a:g}, β={b:g}")
    h_vs_shape = np.array(h_vs_shape)

    # Ploting
    fig, ax = plt.subplots(1, 3, figsize=(12, 3.8), dpi=130)

    ax[0].plot(n_list, h_vs_n, marker="o")
    ax[0].set_title(r"$\hat h_{\mathrm{AMISE}}$ vs sample size $n$")
    ax[0].set_xlabel("n"); ax[0].set_ylabel(r"$\hat h_{\mathrm{AMISE}}$"); ax[0].grid(alpha=.3)

    ax[1].plot(valid_N, h_vs_N, marker="o")
    ax[1].set_title(r"$\hat h_{\mathrm{AMISE}}$ vs block size $N$ (n=800)")
    ax[1].set_xlabel("N"); ax[1].grid(alpha=.3)

    ax[2].bar(range(len(h_vs_shape)), h_vs_shape)
    ax[2].set_xticks(range(len(labels)), labels, rotation=20)
    ax[2].set_title(r"$\hat h_{\mathrm{AMISE}}$ vs Beta$(\alpha,\beta)$ shape (n=800, N=3)")
    ax[2].grid(axis="y", alpha=.3)

    fig.suptitle("Global bandwidth selection (AMISE plug-in, quartic kernel, homoscedastic)", y=1.02, fontsize=11)
    plt.tight_layout()

    
    #plt.savefig("global_bandwidth_selection.png", bbox_inches="tight")
    #plt.close()
    #print("Saved figure as 'global_bandwidth_selection.png'")

In [11]:
if __name__ == "__main__":
    run_experiment(
        alpha_list=(2.0, 2.0, 5.0, 1.5),
        beta_list =(2.0, 5.0, 2.0, 4.0),
        n_list=(200, 400, 800, 1600, 3200),
        N_list=(1, 2, 3, 4, 5, 6),
        sigma=1.0,
        seed=517,
    )

Saved figure as 'global_bandwidth_selection.png'
